<a href="https://colab.research.google.com/github/RubenCarton623/Bot/blob/main/BotDefinitivo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
#pip es un sistema de gestión de paquetes utilizado para instalar y administrar paquetes de software escritos en Python.
!apt update  
!apt install chromium-chromedriver
!pip install selenium

!pip install python-telegram-bot-calendar
!pip install pyTelegramBotAPI

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:5 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:8 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:12 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:14 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:15 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic

In [22]:
from telegram_bot_calendar import DetailedTelegramCalendar, LSTEP
import telebot
from datetime import date
from telebot import types
from bs4 import BeautifulSoup
from time import sleep
from selenium import webdriver
options = webdriver.ChromeOptions()
options.add_argument('--headless')#sin cabecera
options.add_argument('--no-sandbox')#deshabilitar caja de arena
options.add_argument('--disable-dev-shm-usage')#deshabilitar el uso de dev shm

driver = webdriver.Chrome('chromedriver',options=options)#crear una nueva instancia de la clase ChromeDriver

IATA = {"Guayaquil" : "GYE", "Cuenca" : "CUE", "Manta" : "MEC",
		"Coca" : "OCC", "Santa Rosa" : "ETR", "Quito" : "UIO",
		"Baltra" : "GPS", "San Cristobal" : "SCY"}#Lista de palabras reservadas para los vuelos

CLASES = {"Economico":"NA", "Premium Economico":"PE", "Ejecutivo":"C", "Primera Clase":"F"}#lista de palabras reservadas para las clases de vuelos

bot = telebot.TeleBot("1601039476:AAHtnq51buS82tVUeZzFXxJPJL0VjdDvaB4")#token del bot


vuelo_dict = {}#idk.. maik pone weas vacias

class Vuelo:#creacion de la clase vuelo
    def __init__(self, Cliente):#el método init nos ayuda a instanciar objetos de una clase
        self.Cliente = Cliente
        self.Origen = None
        self.Destino = None
        self.Adultos = None
        self.Niños = None
        self.Salida = None
        self.Retorno = None
        self.Tipo = None
        self.Clase = None
        self.EdadN = None
        self.Horas = None
        self.Duracion = None
        self.Escalas = None
        self.HorasR = None
        self.DuracionR = None
        self.EscalasR = None
        self.Precio = None

In [23]:
def Crear_Url(Tipo, Salida, Adultos, Niños, Clase, Origen=None, Destino=None, EdadN = None, Retorno=None):

    if Origen and not Destino:#Si hay origen y no hay destino
        print("Funcion en desarrollo")
    elif Destino and not Origen:#si hay destino y no hay origen
        print("Funcion en desarrollo")
    elif Destino and Origen:#Si hay destino y hay origen
        url = ("https://www.despegar.com.ec/shop/flights/results/{}/{}/{}/{}").format(Tipo,Origen,Destino,Salida)

    if Retorno:#si hay retorno entonces se le agg la fecha de retorno
        url=url+"/"+str(Retorno)

    url=url+"/"+Adultos#Se le agregan los aadultos

    if Niños != 0:#Si existen niños entonces se los agrega
        url=url+"/"
        for i in range(Niños):
            if i == Niños-1:
                url = url + "C"
            else:
                url = url + "C-"
    else:
        url = url + "/0"#LE PONEMOS QUE NO HAY GUAGUAS      

    if not Retorno:#Si no hay retorno entonces
        url = url + "/0/NA/{}/NA/NA?from=SB&di={}-{}".format(Clase, Adultos, Niños)
    else: #Si hay retorno
        url=url+"/0/NA/{}/NA/NA/NA?from=SB&di={}-{}".format(Clase, Adultos, Niños)

    textoEdad=""
    if EdadN:#Si existe edades entonces entonces
        for i in range(len(EdadN)):
            if i == len(EdadN)-1:#ultimo elemento
                textoEdad = textoEdad + EdadN[i]
            else:
                textoEdad = textoEdad + EdadN[i] + "-"#elementos 1, 2, 3.... n-1

    if Niños != 0: #Si existen niños entonces se les agrega la edad a los guaguas
        url = url + ":" + textoEdad

    url = url + "&reSearch=true"#el final de la url GG

    return url

In [24]:
def Scraping(Tipo, Salida, Adultos, Niños, Clase, Origen=None, Destino=None, EdadN = None, Retorno=None):#Ahora si se viene lo shido = scraping
    url = (Crear_Url(Tipo, Salida, Adultos, Niños, Clase, Origen, Destino, EdadN, Retorno))

    print(url)#para comprobar se imprime la url creada :3
    driver.get(url)#le damos la url a la pag creada
    #scraping
    soup = BeautifulSoup(driver.page_source, 'lxml')
    Ofertas = soup.find_all('div', attrs={'class': 'cluster-container COMMON'})
    
    while len(Ofertas)<1:#Mientras no haya llenado las ofertas
        sleep(2)#dormimos dos segundos y scrapeamos de nuevo
        soup = BeautifulSoup(driver.page_source, 'lxml')
        Ofertas = soup.find_all('div', attrs={'class': 'cluster-container COMMON'})
    driver.close()
    #creamos 4 listas donde almacenaremos los datos de salida del vuelo
    HorasS = list()
    DuracionS = list()
    EscalasS = list()
    PrecioF = list()

    if Retorno:#Si hay retorno creamos 4 listas donde almacenaremos los datos de regreso del vuelo
        HorasR = list()
        DuracionR = list()
        EscalasR = list()
        PrecioR = list()

    for i in Ofertas:#Recorremos las ofertas
    #obtener todas las <a>etiquetas que son secundarias de <li>
        Precio = i.findChildren('p', attrs={'class': 'item-fare fare-price'})#obtenemos el precio final
        if not Retorno:#si no hay retorno entonces solo scrapeamos la ida y se le agg un last en el scrapeo por que asi sale en solo ida
            Ida = i.findChildren('span', attrs={'class': 'sub-cluster last'})
        else:#si hay retorno entonces scrapeamos la ida y vuelta

            Ida = [el for el in i.findChildren('span', class_='sub-cluster') if 'last' not in el.attrs['class']]#si no encuentra 'last' en 'class' entonces existe vuelta
            Vuelta = i.findChildren('span', attrs={'class': 'sub-cluster last'})


        for j in Ida:#recorre la ida para llenar los datos
            Horas = j.findChildren('span', attrs={'class': 'hour'})
            Duracion = j.findChildren('span', attrs={'class': 'best-duration' , 'class': 'duration-item-container'})
            Escalas = j.findChildren('span', attrs={'data-sfa-id': 'stops-text'})


            HorasT = list()
            for j in Horas:
                HorasT.append(j.text)#metemos todas las horas en una lista

            DuracionT = list()
            for j in Duracion:
                DuracionT.append(j.text)#metemos todas las duraciones en una lista

            EscalasT = list()
            for j in Escalas:
                if "escala" in j.text.lower():#si escalas está en minuscula
                    if "escalas" in j.text.lower():#si escalas está en minusculas
                        EscalasT.append(j.text.replace(" ","").replace("escalas", ""))
                    else:
                        EscalasT.append(j.text.replace(" ", "").replace("escala", ""))
                elif "directo" in j.text.lower():#si el vuelo es directo entonces no hay escalas
                    EscalasT.append("0")
            #ingresamos todos los vuelos de partida
            HorasS.append(HorasT)
            DuracionS.append(DuracionT)
            EscalasS.append(EscalasT)

        if Retorno:
            for j in Vuelta:
                Horas = j.findChildren('span', attrs={'class': 'hour'})
                Duracion = j.findChildren('span', attrs={'class': 'best-duration'})
                Duracion2 = j.findChildren('span', attrs={'class': 'duration-item-container'})
                Escalas = j.findChildren('span', attrs={'data-sfa-id': 'stops-text'})


                HorasT = list()
                for j in Horas:
                    HorasT.append(j.text)

                DuracionT = list()
                for j in Duracion:
                    DuracionT.append(j.text)

                for j in Duracion2:
                    DuracionT.append(j.text)

                EscalasT = list()
                for j in Escalas:
                    if "escala" in j.text.lower():
                        if "escalas" in j.text.lower():
                            EscalasT.append(j.text.replace(" ","").replace("escalas", ""))
                        else:
                            EscalasT.append(j.text.replace(" ", "").replace("escala", ""))
                    elif "directo" in j.text.lower():
                        EscalasT.append("0")

                HorasR.append(HorasT)
                DuracionR.append(DuracionT)
                EscalasR.append(EscalasT)

        PrecioT = 0
        for j in Precio:
            valor = j.text.replace("Precio final USD", "").replace(".", "")
            if int(valor):
                PrecioT = str(valor)
        PrecioF.append(PrecioT)

    #Si no hay retorno, entonces devolvemos solo ida
    if not Retorno:
        return HorasS, DuracionS, EscalasS, PrecioF
    else:#devolvemos ida y vuelta
        print(PrecioF)
        print(HorasS, DuracionS, EscalasS)
        print(HorasR, DuracionR, EscalasR)
        return HorasS, DuracionS, EscalasS, PrecioF, HorasR, DuracionR, EscalasR,


In [25]:
def listener(messages):
    for message in messages:
        chat_id = message.chat.id
        if(message.text!="/start" and message.text.lower() != "menu"):
            bot.delete_message(chat_id, message.message_id)

@bot.message_handler(regexp='Menu|start')
def Menu(message):

    try:
        chat_id = message.chat.id
        Cliente = message.chat.first_name
        vuelo = Vuelo(Cliente)
        vuelo_dict[chat_id] = Vuelo

        markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
        markup.add("Buscar Vuelos", "Listar Vuelos")
        pregunta = bot.send_message(chat_id, "Selecciona o digita una de las opciones del menu:", reply_markup=markup)

        bot.register_next_step_handler(pregunta, TipoVuelo)
    except Exception as e:
        bot.send_message(chat_id, 'Ops, algo no salio bien... ¡Intentalo de nuevo!')

def TipoVuelo(message):
    try:
        chat_id = message.chat.id
        message_id = message.message_id
        vuelo = vuelo_dict[chat_id]
        vuelo.Salida = ""
        vuelo.Retorno = ""

        opcion = message.text
        if opcion.lower().find("buscar vuelos") == -1:
            bot.delete_message(chat_id, message_id)
            bot.delete_message(chat_id, message_id - 1)
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
            markup.add("Buscar Vuelos", "Listar Vuelos")
            pregunta = bot.send_message(chat_id, "¡Seleccione una opcion valida!", reply_markup=markup)
            bot.register_next_step_handler(pregunta, TipoVuelo)
            return

        bot.delete_message(chat_id, message_id)
        bot.delete_message(chat_id, message_id - 1)

        markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
        markup.add("Ida", "Ida y vuelta")
        pregunta = bot.send_message(chat_id, 'Seleccione o digite el tipo de vuelo:', reply_markup=markup)

        bot.register_next_step_handler(pregunta, Origen)
    except Exception as e:
        bot.send_message(chat_id, 'Ops, parece que tuvimos un problema...')

def Origen(message):
    try:
        chat_id = message.chat.id
        message_id = message.message_id
        vuelo = vuelo_dict[chat_id]

        opcion = message.text.lower()
        if opcion == "ida":
            opcion = "oneway"
        elif opcion == "ida y vuelta":
            opcion = "roundtrip"

        if opcion!="oneway" and opcion!="roundtrip":
            bot.delete_message(chat_id, message_id)
            bot.delete_message(chat_id, message_id - 1)
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
            markup.add("Ida", "Ida y vuelta")
            pregunta = bot.send_message(chat_id, "¡Seleccione una tipo de vuelo valido!", reply_markup=markup)
            bot.register_next_step_handler(pregunta, Origen)
            return

        vuelo.Tipo=opcion
        bot.delete_message(chat_id, message_id)
        bot.delete_message(chat_id, message_id - 1)

        markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
        markup.add("Guayaquil", "Cuenca", "Manta", "Coca", "Santa Rosa", "Quito", "Baltra" ,"San Cristobal")
        pregunta = bot.send_message(chat_id, 'Seleccione o digite el lugar de origen:', reply_markup=markup)

        bot.register_next_step_handler(pregunta, Destino)
    except Exception as e:
        bot.send_message(chat_id, 'Ops, parece que tuvimos un problema...')



def Destino(message):
    try:
        chat_id = message.chat.id
        message_id = message.message_id
        vuelo = vuelo_dict[chat_id]

        origen = message.text
        if str(IATA).find(origen) == -1:
            bot.delete_message(chat_id, message_id)
            bot.delete_message(chat_id, message_id - 1)
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
            markup.add("Guayaquil", "Cuenca", "Manta", "Coca", "Santa Rosa", "Quito", "Baltra", "San Cristobal")
            pregunta = bot.send_message(chat_id, '¡Digite un origen valido!', reply_markup=markup)
            bot.register_next_step_handler(pregunta, Destino)
            return

        bot.delete_message(chat_id, message_id)
        bot.delete_message(chat_id, message_id - 1)
        bot.send_message(chat_id,"Origen: "+ origen)

        vuelo.Origen = IATA.get(origen)

        markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
        markup.add("Guayaquil", "Cuenca", "Manta", "Coca", "Santa Rosa", "Quito", "Baltra", "San Cristobal")
        pregunta = bot.send_message(chat_id, 'Seleccione o digite el lugar de destino:', reply_markup=markup)

        bot.register_next_step_handler(pregunta, Adultos)
    except Exception as e:
        bot.send_message(chat_id, 'Ops, parece que tuvimos un problema...')

def Adultos(message):
    try:
        chat_id = message.chat.id
        message_id = message.message_id
        vuelo = vuelo_dict[chat_id]

        destino = message.text
        if str(IATA).find(destino) == -1:
            bot.delete_message(chat_id, message_id)
            bot.delete_message(chat_id, message_id - 1)
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
            markup.add("Guayaquil", "Cuenca", "Manta", "Coca", "Santa Rosa", "Quito", "Baltra", "San Cristobal")
            pregunta = bot.send_message(chat_id, '¡Digite un destino valido!', reply_markup=markup)
            bot.register_next_step_handler(pregunta, Fecha)
            return

        bot.delete_message(chat_id, message_id)
        bot.delete_message(chat_id, message_id - 1)
        bot.send_message(chat_id, "Destino: " + destino)
        vuelo.Destino = IATA.get(destino)

        markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
        markup.add("1", "2", "3", "4", "5", "6", "7", "8")
        pregunta = bot.send_message(chat_id, 'Seleccione o digite cuantos adultos van a viajar:', reply_markup=markup)

        bot.register_next_step_handler(pregunta, Niños)
    except Exception as e:
        bot.send_message(chat_id, 'Ops, parece que tuvimos un problema...')

def Niños(message):
    # try:
        chat_id = message.chat.id
        message_id = message.message_id
        vuelo = vuelo_dict[chat_id]

        adultos = message.text

        if not adultos.isdigit():
            if adultos == "+12 (Tasa de adulto)":
                adultos = "12"
            else:
                adultos = "0"

        if int(adultos)<1 or int(adultos)>8:
                bot.delete_message(chat_id, message_id)
                bot.delete_message(chat_id, message_id - 1)
                markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
                markup.add("1", "2", "3", "4", "5", "6", "7", "8")
                pregunta = bot.send_message(chat_id, 'Seleccione o digite una cantidad valida:', reply_markup=markup)
                bot.register_next_step_handler(pregunta, Niños)
                return

        bot.delete_message(chat_id, message_id)
        bot.delete_message(chat_id, message_id - 1)
        bot.send_message(chat_id, "Adultos: " + adultos)
        vuelo.Adultos = adultos

        markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
        for i in range(9 - int(vuelo.Adultos)):
            markup.add(str(i))
        pregunta = bot.send_message(chat_id, 'Seleccione o digite cuantos niños van a viajar:', reply_markup=markup)

        bot.register_next_step_handler(pregunta, Fecha)
    # except Exception as e:
    #     bot.send_message(chat_id, 'Ops, parece que tuvimos un problema...')

def Fecha(message):
    # try:
        chat_id = message.chat.id
        message_id = message.message_id
        vuelo = vuelo_dict[chat_id]



        niños = message.text

        if not niños.isdigit():
            niños = "10"

        if int(niños) < 0 or int(niños) > 9-int(vuelo.Adultos):
                bot.delete_message(chat_id, message_id)
                bot.delete_message(chat_id, message_id - 1)
                markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
                for i in range (8-int(vuelo.Adultos)):
                    markup.add(str(i))
                pregunta = bot.send_message(chat_id, 'Seleccione o digite una cantidad valida:', reply_markup=markup)
                bot.register_next_step_handler(pregunta, Fecha)
                return

        vuelo.Niños = niños
        vuelo.EdadN = list()


        if niños == "0":
            bot.delete_message(chat_id, message_id)
            bot.delete_message(chat_id, message_id - 1)
            bot.send_message(chat_id, "Niños: " + niños)

            calendar, step = DetailedTelegramCalendar(min_date=date.today(), max_date=date(date.today().year, date.today().month, date.today().day)).build()
            pregunta = bot.send_message(message.chat.id, f"Select {LSTEP[step]}", reply_markup=calendar)
        else:
            bot.delete_message(chat_id, message_id)
            bot.delete_message(chat_id, message_id - 1)
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
            markup.add("1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "+12 (Tasa de adulto)")
            pregunta = bot.send_message(chat_id, 'Edad del 1° niño: ', reply_markup=markup)

            bot.register_next_step_handler(pregunta, seleccionarEdades)

    # except Exception as e:
    #     bot.send_message(chat_id, 'Ops, parece que tuvimos un problema...')

def seleccionarEdades(message):
    #try:


        chat_id = message.chat.id
        message_id = message.message_id
        vuelo = vuelo_dict[chat_id]

        edad = message.text

        if not edad.isdigit():
            if edad == "+12 (Tasa de adulto)":
                edad = "12"
            else:
                edad = "0"

        if int(edad) < 1 or int(edad) > 12:
                bot.delete_message(chat_id, message_id)
                bot.delete_message(chat_id, message_id - 1)
                markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
                markup.add("1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "+12 (Tasa de adulto)")
                pregunta = bot.send_message(chat_id, 'Edad del ' + str(len(vuelo.EdadN)+1) + "° niño:" , reply_markup=markup)
                bot.register_next_step_handler(pregunta, seleccionarEdades)
                return

        vuelo.EdadN.append(edad)

        if len(vuelo.EdadN) == int(vuelo.Niños):
            bot.delete_message(chat_id, message_id)
            bot.delete_message(chat_id, message_id - 1)

            texto="Niños: " + vuelo.Niños + "\n"
            for i in vuelo.EdadN:
                if int(i) == 1:
                    texto=texto+"- Tarifa bebé\n"
                if int(i) == 12:
                    texto=texto+"- Tarifa adulto\n"
                if int(i) < 12 and int(i) > 1:
                    texto=texto+"- Tarifa niño\n"

            bot.send_message(chat_id, texto)

            calendar, step = DetailedTelegramCalendar(min_date=date.today(),
                                                      max_date=date(date.today().year, date.today().month,
                                                                    date.today().day)).build()
            pregunta = bot.send_message(message.chat.id, f"Select {LSTEP[step]}", reply_markup=calendar)
        else:
            bot.delete_message(chat_id, message_id)
            bot.delete_message(chat_id, message_id - 1)
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
            markup.add("1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "+12 (Tasa de adulto)")
            pregunta = bot.send_message(chat_id, 'Edad del ' + str(len(vuelo.EdadN)+1) + "° niño:", reply_markup=markup)
            bot.register_next_step_handler(pregunta, seleccionarEdades)

    #except Exception as e:
        #bot.send_message(chat_id, 'Ops, parece que tuvimos un problema...')

def Clase(message):
    # try:
        chat_id = message.chat.id
        message_id = message.message_id
        vuelo = vuelo_dict[chat_id]
        clase = message.text

        if str(CLASES).find(clase) == -1:
            bot.delete_message(chat_id, message_id)
            bot.delete_message(chat_id, message_id - 1)
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
            markup.add("Economico", "Premium Economico", "Ejecutivo", "Primera Clase")
            pregunta = bot.send_message(chat_id, '¡Digite una clase valida!', reply_markup=markup)
            bot.register_next_step_handler(pregunta, Clase)
            return

        bot.delete_message(chat_id, message_id)
        bot.delete_message(chat_id, message_id - 1)
        bot.send_message(chat_id, "Clase: " + clase)

        pregunta = bot.send_message(chat_id, 'Estoy buscando los mejores vuelos para ti...')

        BuscarVuelo(message)
    # except Exception as e:
    #     pregunta = bot.send_message(chat_id, 'Ops, parece que tuvimos un problema...')

def NoVuelos(message):
    # try:
        chat_id = message.chat.id
        message_id = message.message_id
        vuelo = vuelo_dict[chat_id]

        opcion = message.text

        if "cambiar fecha|cambiar clase|buscar otro vuelo|volver al menu".find(opcion.lower()) == -1:
            bot.delete_message(chat_id, message_id)
            bot.delete_message(chat_id, message_id - 1)
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
            markup.add("Cambiar fecha", "Cambiar clase", "Buscar otro vuelo", "Volver al menu")
            pregunta = bot.send_message(chat_id, '¡Seleccione una opcion valida!', reply_markup=markup)
            bot.register_next_step_handler(pregunta, NoVuelos)
            return

        bot.delete_message(chat_id, message_id)
        bot.delete_message(chat_id, message_id - 1)

        if opcion.lower() == "cambiar fecha":
            calendar, step = DetailedTelegramCalendar(min_date=date.today(),
                                                      max_date=date(date.today().year, date.today().month,
                                                                    date.today().day)).build()
            pregunta = bot.send_message(message.chat.id, f"Select {LSTEP[step]}", reply_markup=calendar)

        elif opcion.lower() == "cambiar clase":
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
            markup.add("Economico", "Premium Economico", "Ejecutivo", "Primera Clase")
            pregunta = bot.send_message(chat_id, 'Seleccione en que clase desea viajar:', reply_markup=markup)

            bot.register_next_step_handler(pregunta, Clase)

        elif opcion.lower() == "buscar otro vuelo":
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
            markup.add("Guayaquil", "Cuenca", "Manta", "Coca", "Santa Rosa", "Quito", "Baltra", "San Cristobal")
            pregunta = bot.send_message(chat_id, 'Seleccione o digite el lugar de origen:', reply_markup=markup)

            bot.register_next_step_handler(pregunta, Destino)

        elif opcion.lower() == "volver al menu":
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
            markup.add("Buscar Vuelos", "Listar Vuelos")
            pregunta = bot.send_message(chat_id, "Selecciona o digita una de las opciones del menu:",
                                        reply_markup=markup)

            bot.register_next_step_handler(pregunta, Origen)

    # except Exception as e:
    #     bot.send_message(chat_id, 'Ops, parece que tuvimos un problema...')

def BuscarVuelo(message):
    chat_id = message.chat.id
    message_id = message.message_id
    vuelo = vuelo_dict[chat_id]

    clase = message.text
    vuelo.Clase = CLASES.get(clase)


    if vuelo.Retorno=="":
        vuelo.Horas, vuelo.Duracion, vuelo.Escalas, vuelo.Precio = Scraping(vuelo.Tipo, vuelo.Salida, vuelo.Adultos, int(vuelo.Niños), vuelo.Clase, vuelo.Origen, vuelo.Destino, vuelo.EdadN)
    else:
        vuelo.Horas, vuelo.Duracion, vuelo.Escalas, vuelo.Precio, vuelo.HorasR, vuelo.DuracionR, vuelo.EscalasR = Scraping(vuelo.Tipo, vuelo.Salida, vuelo.Adultos, int(vuelo.Niños), vuelo.Clase, vuelo.Origen, vuelo.Destino, vuelo.EdadN, vuelo.Retorno)

    bot.delete_message(chat_id, message_id+2)

    if len(vuelo.Precio)>0:
        bot.send_message(chat_id, "¡Listo! He encontrado los siguientes vuelos:")

        for i in range(0, len(vuelo.Precio)):
            message = "*\-\-\-\- Salida \-\-\-\-*\n"
            markup = types.InlineKeyboardMarkup()
            markup_button = types.InlineKeyboardButton("$" + vuelo.Precio[i], callback_data=str(i))
            markup.row(markup_button)

            contador = 0
            for j in range(0, len(vuelo.Escalas[i])):
                message = message + "*" + vuelo.Origen + "* _" + vuelo.Horas[i][contador] + "_ \- *" + vuelo.Destino + "* _" + \
                          vuelo.Horas[i][contador + 1] + "_\n*\- Duracion:* _" + vuelo.Duracion[i][
                              j] + "_ " + "*\n\- Escalas:* _" + vuelo.Escalas[i][j] + "_\n"
                contador = contador + 2

            if vuelo.Retorno!="":
                message=message+"*\-\-\-\- Retorno \-\-\-\-*\n"
                contador=0
                for j in range(0, len(vuelo.EscalasR[i])):

                    message = message + "*" + vuelo.Origen + "* _" + vuelo.HorasR[i][
                        contador] + "_ \- *" + vuelo.Destino + "* _" + \
                              vuelo.HorasR[i][contador + 1] + "_\n*\- Duracion:* _" + vuelo.DuracionR[i][
                                  j] + "_ " + "*\n\- Escalas:* _" + vuelo.EscalasR[i][j] + "_\n"
                    contador = contador + 2

            bot.send_message(chat_id, message, parse_mode="MarkdownV2", reply_markup=markup, disable_web_page_preview=True)
    else:
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
        markup.add("Cambiar fecha", "Cambiar clase", "Buscar otro vuelo", "Volver al menu")
        pregunta = bot.send_message(chat_id, 'No se encontraron vuelos en la fecha o clase seleccionada... ¿Que deberiamos hacer?', reply_markup=markup)

        bot.register_next_step_handler(pregunta, NoVuelos)

def HoraSalida(message):
    chat_id = message.chat.id
    message_id = message.message_id
    vuelo = vuelo_dict[chat_id]

    salida = message.text
    valida = False
    if salida.find("-") != -1:
       salidaT = salida.replace("-","").replace(":","").replace(" ","")
       if salidaT.isdigit() and len(salidaT)== 4:
            valida=True

    if valida==False:
        bot.delete_message(chat_id, message_id)
        i = vuelo.Oferta
        markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)

        contador = 0
        for j in range(0, len(vuelo.Escalas[i])):
            markup.add(vuelo.Horas[i][contador] + " - " + vuelo.Horas[i][contador+1])
            contador = contador + 2

        pregunta = bot.send_message(chat_id, "Seleccione la hora de salida:", reply_markup=markup)
        bot.register_next_step_handler(pregunta, HoraSalida)
        return

    pregunta = bot.send_message(chat_id, "Cool")


@bot.callback_query_handler(func=lambda call: True)
def cal(c):
    chat_id = c.message.chat.id
    if not c.data.isdigit():
        result, key, step = DetailedTelegramCalendar(min_date=date.today(), max_date=date(date.today().year+1, date.today().month, date.today().day)).process(c.data)

        if not result and key:
            bot.edit_message_text(f"Select {LSTEP[step]}",
                                  c.message.chat.id,
                                  c.message.message_id,
                                  reply_markup=key)
        elif result:
            bot.delete_message(c.message.chat.id, c.message.message_id)


            vuelo = vuelo_dict[chat_id]
            if vuelo.Tipo=="oneway":
                bot.send_message(chat_id, "Fecha de partida: " + str(result))

                markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
                markup.add("Economico", "Premium Economico", "Ejecutivo", "Primera Clase")
                pregunta = bot.send_message(chat_id, 'Seleccione en que clase desea viajar:', reply_markup=markup)

                vuelo.Salida = result
                bot.register_next_step_handler(pregunta, Clase)

            else:
                if vuelo.Salida=="":
                    bot.send_message(chat_id, "Fecha de partida: " + str(result))
                    vuelo.Salida = result

                    calendar, step = DetailedTelegramCalendar(min_date=date.today(),
                                                              max_date=date(date.today().year, date.today().month,
                                                                            date.today().day)).build()
                    pregunta = bot.send_message(chat_id, f"Select {LSTEP[step]}", reply_markup=calendar)
                else:
                    bot.send_message(chat_id, "Fecha de retorno: " + str(result))
                    vuelo.Retorno = result

                    markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)
                    markup.add("Economico", "Premium Economico", "Ejecutivo", "Primera Clase")
                    pregunta = bot.send_message(chat_id, 'Seleccione en que clase desea viajar:', reply_markup=markup)

                    bot.register_next_step_handler(pregunta, Clase)


    else:
        #try:
            i = int(c.data)
            vuelo = vuelo_dict[chat_id]
            vuelo.Oferta = i

            for j in range(0, len(vuelo.Escalas)):
                 bot.delete_message(c.message.chat.id, c.message.message_id - j + len(vuelo.Escalas) - i - 1)

            message = "*\-\-\-\- Salida \-\-\-\-*\n"
            markup = types.ReplyKeyboardMarkup(resize_keyboard=True, one_time_keyboard=True)


            contador = 0
            for j in range(0, len(vuelo.Escalas[i])):
                markup.add(vuelo.Horas[i][contador] + " - " + vuelo.Horas[i][contador+1])
                message = message + "*" + vuelo.Origen + "* _" + vuelo.Horas[i][contador] + "_ \- *" + vuelo.Destino + "* _" + \
                          vuelo.Horas[i][contador + 1] + "_\n*\- Duracion:* _" + vuelo.Duracion[i][
                              j] + "_ " + "*\n\- Escalas:* _" + vuelo.Escalas[i][j] + "_\n"
                contador = contador + 2

            if vuelo.Retorno!="":
                message=message+"*Retorno:*\n"
                contador=0
                for j in range(0, len(vuelo.EscalasR[i])):
                    #markup.add(vuelo.HorasR[i][contador] + " - " + vuelo.HorasR[i][contador])
                    message = message + "*" + vuelo.Origen + "* _" + vuelo.HorasR[i][
                        contador] + "_ \- *" + vuelo.Destino + "* _" + \
                              vuelo.HorasR[i][contador + 1] + "_\n*\- Duracion:* _" + vuelo.DuracionR[i][
                                  j] + "_ " + "*\n\- Escalas:* _" + vuelo.EscalasR[i][j] + "_\n"
                    contador = contador + 2

            pregunta = bot.send_message(chat_id, message, parse_mode="MarkdownV2", reply_markup=markup)
            bot.register_next_step_handler(pregunta, HoraSalida)
        #except Exception as e:
            #print("Error: Boton en linea")


bot.enable_save_next_step_handlers(delay=2)
bot.load_next_step_handlers()
bot.set_update_listener(listener)
bot.polling()

https://www.despegar.com.ec/shop/flights/results/oneway/GYE/CUE/2021-03-10/1/0/0/NA/NA/NA/NA?from=SB&di=1-0&reSearch=true
